使用許諾条件

①	設計一次エネルギー消費量算定法に基づく計算方法をPythonによりコーディングし、Jupyter notebook形式で保存したファイル（以下、本ファイルと呼ぶ）の著作権は、一般社団法人日本サステナブル建築協会に帰属します。
Copyright©2019. 一般社団法人日本サステナブル建築協会
All rights reserved. 

②	本ファイルは、住戸の設計一次エネルギー消費量算定方法に係る「平成28年省エネルギー基準に準拠したエネルギー消費性能の評価に関する技術情報（住宅）2.1算定方法（第四章第六節 電気蓄熱暖房器）」に示す計算方法に基づき、国土交通省国土技術政策総合研究所・国立研究開発法人建築研究所、株式会社ユーワークス、株式会社住環境計画研究所が作成したものです。

③	本ファイルは、計算方法に示す計算過程の確認及び調査研究等に活用することを目的として作成されたものであり、一般社団法人日本サステナブル建築協会は著作権について放棄はしませんが、使用する各人の自己責任において仕様を変更することを妨げるものではありません。ただし、本ファイルによる計算結果を住宅の省エネルギー基準で規定する設計一次エネルギー消費量の結果として公的な申請に用いることはできません。

④	本ファイルを変更するかしないかを問わず、以下の条件を満たす場合に限り、頒布および使用が許可されます。
1.	頒布物の付属ドキュメント等の資料に、上記の著作権表示、本条件一覧、および下記免責条項を含めること。
2.	書面による特別の許可なしに、本ファイルから派生した製品の宣伝または販売の促進に、一般社団法人日本サステナブル建築協会、国土技術政策総合研究所・国立研究開発法人建築研究所および株式会社ユーワークス、株式会社住環境計画研究所の名前またはコントリビューターの名前を使用してはならない。

⑤	本ファイルは、著作権者およびコントリビューターによって「現状のまま」提供されており、明示黙示を問わず、商業的な使用可能性、および特定の目的に対する適合性に関する暗黙の保証も含め、またそれに限定されない、いかなる保証もありません。著作権もコントリビューターも、事由のいかんを問わず、損害発生の原因いかんを問わず、かつ責任の根拠が契約であるか厳格責任であるか（過失その他の）不法行為であるかを問わず、仮にそのような損害が発生する可能性を知らされていたとしても、本ファイルの使用によって発生した（代用品または代用サービスの調達、使用の喪失、データの喪失、利益の喪失、業務の中断も含め、またそれに限定されない）直接損害、間接損害、偶発的な損害、特別損害、懲罰的損害、または結果損害について、一切の責任を負わないものとします。



In [ ]:
import nbimporter
import numpy as np
from s04_01_general import get_Q_T_H_d_t_i

## 第六節　電気蓄熱暖房器 / Section 6 Electric thermal storage type heating

1．適用範囲～4．記号及び単位は省略

1. Scope of application ~ 4. Symbols and units are omitted

### 5．最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,d,t} = q_{rtd,H} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h) / Maximum heating output per hour at time $ t $ of date $ d $  
$q_{rtd,H}$：定格暖房能力(W) / Rated heating capacity

In [ ]:
def get_Q_max_H_d_t(q_rtd_H):
    return np.ones(24 * 365) * (q_rtd_H * 3600 / 1000000)  # (1)

### 6．暖房エネルギー消費量 / Heating energy consumption

#### 6.1　消費電力量 / Power consumption

$$
\large
E_{E,H,d,t} = Q_{T,H,d,t} \times \frac{1}{e_{rtd,H}} \times 10^3 \div 3600
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h) / Power consumption per hour at time $ t $ of date $ d $  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h) / Processing heating load per hour at time $ t $ of date $ d $  
$e_{rtd,H}$：蓄熱効率 / Heat storage efficiency 

In [ ]:
def get_E_E_H_d_t(q_rtd_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_rtd_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    return Q_T_H_d_t * (1.0 / e_rtd_H) * 1000 / 3600  # (2)


def get_Q_UT_H_d_t(q_rtd_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_rtd_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    return L_H_d_t - Q_T_H_d_t

#### 6.2　ガス消費量 / Gas consumption

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は0とする。

It is assumed that gas consumption $ E_{G, H, d, t} $ per hour at time $ t $ of date $ d $ is zero.

In [ ]:
def get_E_G_H_d_t():
    return np.zeros(24 * 365)

#### 6.3　灯油消費量 / Kerosene consumption

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は0とする。

It is assumed that kerosene consumption $ E_{K, H, d, t} $ per hour at time $ t $ of date $ d $ is zero.

In [ ]:
def get_E_K_H_d_t():
    return np.zeros(24 * 365)

#### 6.4　その他の燃料による一次エネルギー消費量 / Primary energy consumption by other fuels

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は0とする。

Primary energy consumption $ E_{M, H, d, t} $ by other fuel per hour at time $ t $ of date $ d $ is 0.

In [ ]:
def get_E_M_H_d_t():
    return np.zeros(24 * 365)

## 付録 A 機器の性能を表す仕様の決定方法 / Appendix A How to determine the specification that represents the performance of the device

### A.1　記号及び単位 / Symbol and unit

省略

ommision

### A.2　定格暖房能力 / Rated heating capacity

$$\large
q_{rtd,H} = q_{rq,H} \times A_{HCZ} \times f_{cT} \times f_{cI}
$$

$q_{rtd,H}$：定格暖房能力(kWh/h) / Rated heating capacity  
$q_{rq,H}$：単位面積当たりの必要暖房能力(W/m<sup>2</sup>) / Required heating capacity per unit area  
$A_{HCZ}$：電気蓄熱暖房器により暖房される暖冷房区画の床面積(m<sup>2</sup>) / Floor area of heating and cooling compartments heated by electric heat storage heater  
$f_{cT}$：外気温度能力補正係数 / Outside air temperature ability correction factor  
$f_{cI}$：間歇運転能力補正係数 / Intermittent driving ability correction factor  

外気温度補正係数$f_{cT}$の値は1.05とする。

The value of the outside air temperature correction coefficient $ f_{cT} $ is set to 1.05.

In [ ]:
def get_q_rtd_H(q_rq_H, A_HCZ, f_cT, f_cI):
    return q_rq_H * A_HCZ * f_cT * f_cI  # (1)


# 単位面積当たりの必要暖房能力
def get_q_rq_H(region):
    return table_a_2[region - 1]


# 外気温度補正係数
def get_f_cT(region):
    return 1.05


# 間歇運転能力補正係数
def get_f_cI(mode, R_type):
    if mode in ['ろ', '連続']:
        y = 0
    elif mode in ['は', '間歇']:
        y = 1
    else:
        raise ValueError(mode)

    if R_type == '主たる居室':
        x = 0
    elif R_type == 'その他の居室':
        x = 1
    else:
        raise ValueError(R_type)

    return table_a_3[y][x]

<div style="text-align: center;font-weight: bold;">表　単位面積当たりの必要暖房能力（W/m<sup>2</sup>）<br>Table: Required heating capacity per unit area</div>

|地域の区分<br>Segmentation of area|1|2|3|4|5|6|7|8|
|-|-|-|-|-|-|-|-|-|
|　|139.3|120.7|111.3|119.0|126.6|106.5|112.9|-|

In [ ]:
# 表A.2 単位面積当たりの必要暖房能力
table_a_2 = [
    139.3,
    120.7,
    111.3,
    119.0,
    126.6,
    106.5,
    112.9
]

<div style="text-align: center;font-weight: bold;">表　間歇運転能力補正係数<br>Table: Intermittent driving ability correction factor</div>

|　|主たる居室<br>Main room|その他の居室<br>Other rooms|
|-|-|-|
|連続運転の場合<br>In case of continuous operation|1.0|1.0|
|間歇運転の場合<br>In case of intermittent operation|3.034|4.805|

In [ ]:
# 表A.3 間歇運転能力補正係数
table_a_3 = [
    (1.0, 1.0),
    (3.034, 4.805)
]

### A.3　蓄熱効率 / Heat storage efficiency 

蓄熱効率$e_{rtd.H}$は0.850（85.0%）とする。 

The heat storage efficiency $ e_{rtd. H} $ is 0.850 (85.0%).

In [ ]:
def get_e_rtd_H():
    return 0.850